In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import os
import finnhub
import pandas
from datetime import datetime, timedelta
import yfinance as yf
import re


In [2]:
from selenium import webdriver

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
path = "/Users/lukasamare/Desktop/chromedriver-mac-arm64/chromedriver"

service= Service(executable_path=path)

In [3]:
url =  "https://finance.yahoo.com/screener/predefined/undervalued_growth_stocks/"
options = Options()
options.binary_location = "/Users/lukasamare/Desktop/chrome-mac-arm64/Google Chrome for Testing.app/Contents/MacOS/Google Chrome for Testing"
driver = webdriver.Chrome(service = service, options = options)
#options.binary_location = os.path.expanduser("~/Desktop/Google Chrome.app/Contents/MacOS/Google Chrome")
driver.get(url)
element  = driver.find_elements(By.XPATH, "//*[@data-test='quoteLink']")
ticker_lst = []
for ticker in element:
    ticker_lst.append(ticker.text)
print(ticker_lst)

driver.quit()


['DAL', 'CVE', 'LYG', 'LAUR', 'AEO', 'BKR', 'CVS', 'MTCH', 'JWN', 'MGM', 'SYF', 'DOLE', 'NOV', 'HMY', 'GS', 'FOXA', 'APLE', 'URBN', 'SLM', 'EGO', 'ACGL', 'BBVA', 'FNF', 'ARCC', 'CHX']


In [4]:
print(len(ticker_lst))

25


In [5]:

finnhub_client = finnhub.Client(api_key="crq5k7pr01qutsn3h690crq5k7pr01qutsn3h69g")


In [6]:
today = datetime.today()

two_years = today -timedelta(days=365*2)
two_months_ago = today-timedelta(days=60)
two_months = str(two_months_ago).replace("-","").replace(" ","T")[:14].replace(":","")

print(two_months)

20240805T1113


In [7]:
today = str(today).split()[0]
two_years = str(two_years).split()[0]
d = finnhub_client.stock_insider_transactions("DAL",two_years,today)
#filters data into a data frame  with top 3 transaction values
def filter_data_trans(data):
  pos_trans = [elem for elem in data["data"] if elem["change"]>0]
  df = pd.DataFrame(pos_trans)
 
  df["transaction_value"]= df["change"]*df["transactionPrice"]
  top3 = df["transaction_value"].sort_values(ascending=False)[:3]
  filt = df[df["transaction_value"].isin(top3)]
  

  return filt
filter_data_trans(d)
d

{'data': [{'change': -17445,
   'currency': '',
   'filingDate': '2024-07-15',
   'id': '0001683168-24-004853',
   'isDerivative': False,
   'name': 'Janki Daniel C.',
   'share': 187172,
   'source': 'sec',
   'symbol': 'DAL',
   'transactionCode': 'F',
   'transactionDate': '2024-07-12',
   'transactionPrice': 43.62},
  {'change': 10000,
   'currency': '',
   'filingDate': '2024-07-15',
   'id': '0001683168-24-004850',
   'isDerivative': False,
   'name': 'Chiang Willie CW',
   'share': 20000,
   'source': 'sec',
   'symbol': 'DAL',
   'transactionCode': 'P',
   'transactionDate': '2024-07-12',
   'transactionPrice': 43.8967},
  {'change': 4040,
   'currency': '',
   'filingDate': '2024-06-21',
   'id': '0001683168-24-004413',
   'isDerivative': False,
   'name': 'Black Maria',
   'share': 4055,
   'source': 'sec',
   'symbol': 'DAL',
   'transactionCode': 'A',
   'transactionDate': '2024-06-20',
   'transactionPrice': 0},
  {'change': 4040,
   'currency': '',
   'filingDate': '2024-

In [8]:
#merges transactions into one dataframe
df_lst = []
for ticks in ticker_lst:
    call = finnhub_client.stock_insider_transactions(ticks,two_years,today)
    if call["data"]==[]:
       print(ticks)
    else:
        df_lst.append(filter_data_trans(call))
        top3_instrans_alltickers = pd.concat(df_lst,ignore_index=True)
#push this data frame into snowflake
top3_instrans_alltickers

#make dataframe where it shows ticker and max transaction over 200,000 
#category column with all other data 


LYG
DOLE
HMY


/var/folders/8l/kydfwjds0pdf_9nnjjth36t80000gn/T/ipykernel_63366/842345839.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  top3_instrans_alltickers = pd.concat(df_lst,ignore_index=True)
/var/folders/8l/kydfwjds0pdf_9nnjjth36t80000gn/T/ipykernel_63366/842345839.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  top3_instrans_alltickers = pd.concat(df_lst,ignore_index=True)
/var/folders/8l/kydfwjds0pdf_9nnjjth36t80000gn/T/ipykernel_63366/842345839.py:9: FutureWarning: The behavior of DataFram

,change,currency,filingDate,id,isDerivative,name,share,source,symbol,transactionCode,transactionDate,transactionPrice,transaction_value
0,10000,,2024-05-03,0001683168-24-002965,False,Chiang Willie CW,10000.0,sec,DAL,P,2024-05-01,49.4955,494955.00
1,71840,,2024-02-02,0001683168-24-000652,False,Bastian Edward H,500608.0,sec,DAL,M,2024-02-01,30.8900,2219137.60
2,12880,,2023-01-20,0001683168-23-000263,False,BLAKE FRANCIS S,93795.0,sec,DAL,P,2023-01-18,38.5795,496903.96
3,3233900,CAD,2024-09-10,,None,Cenovus Energy Inc.,3233900.0,sedi,CVE.TO,38,2024-08-20,25.8500,83596315.00
4,1912700,CAD,2023-01-10,,None,Cenovus Energy Inc.,1912700.0,sedi,CVE.TO,38,2022-12-05,26.3400,50380518.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,3000,,2023-03-17,0001104659-23-033966,False,ROLL PENELOPE F,60500.0,sec,ARCC,P,2023-03-16,17.4400,52320.00
253,3000,,2023-03-17,0001104659-23-033966,False,ROLL PENELOPE F,57500.0,sec,ARCC,P,2023-03-15,17.5000,52500.00
254,20000,,2023-09-11,0001723089-23-000149,False,Somasundaram Sivasankaran,475499.0,sec,CHX,M,2023-09-07,34.1300,682600.00
255,39264,,2022-11-15,0001723089-22-000156,False,Somasundaram Sivasankaran,411991.0,sec,CHX,M,2022-11-11,24.6500,967857.60


In [9]:
tvh_ticker_lst = []
for i in range(len(top3_instrans_alltickers)):
    if top3_instrans_alltickers.iloc[i]["transaction_value"] >200000:
        tvh_ticker_lst.append(top3_instrans_alltickers.iloc[i]["symbol"])
print()       
tvh_ticker_lst = list(set(tvh_ticker_lst))
print(tvh_ticker_lst)  






['SLM', 'AEO', 'MGM', 'CVS', 'FOXA', 'ACGL', 'CVE.TO', 'SYF', 'DAL', 'CHX', 'ELD.TO', 'BBVA.MC', 'BKR', 'APLE', 'MTCH', 'JWN', 'URBN', 'ARCC', 'FNF']


In [10]:
import re
path = "/Users/lukasamare/Desktop/chromedriver-mac-arm64/chromedriver"
service= Service(executable_path=path)
options = Options()
options.binary_location = "/Users/lukasamare/Desktop/chrome-mac-arm64/Google Chrome for Testing.app/Contents/MacOS/Google Chrome for Testing"
driver = webdriver.Chrome(service = service, options = options)

urllst = [ "https://finance.yahoo.com/quote/"+ticker+"/key-statistics/" for ticker in tvh_ticker_lst]
print(urllst)
sir_data = []
#getting all the urls in a list from the cleaned insider transaction ticker list
for ticker in tvh_ticker_lst:
    #getting all the urls in a list from the cleaned insider transaction ticker list
    print(ticker)
    url = "https://finance.yahoo.com/quote/"+ticker+"/key-statistics/"
    driver.get(url)
    element  = driver.find_elements(By.XPATH, "//td[@class='value yf-vaowmx']")
    statslst = []
    #appending all the stats from all the rows int a list 
    for stats in element:
         statslst.append(stats.text)
    # print(statslst)
    shortlst ={"Ticker": ticker}
    if re.match(r"^\d", statslst[37]):
      shortlst["Short_Interest_Rate"] = float(statslst[37])
    else:
        shortlst["Short_Interest_Rate"] = None
    sir_data.append(shortlst)

driver.quit()
    


['https://finance.yahoo.com/quote/SLM/key-statistics/', 'https://finance.yahoo.com/quote/AEO/key-statistics/', 'https://finance.yahoo.com/quote/MGM/key-statistics/', 'https://finance.yahoo.com/quote/CVS/key-statistics/', 'https://finance.yahoo.com/quote/FOXA/key-statistics/', 'https://finance.yahoo.com/quote/ACGL/key-statistics/', 'https://finance.yahoo.com/quote/CVE.TO/key-statistics/', 'https://finance.yahoo.com/quote/SYF/key-statistics/', 'https://finance.yahoo.com/quote/DAL/key-statistics/', 'https://finance.yahoo.com/quote/CHX/key-statistics/', 'https://finance.yahoo.com/quote/ELD.TO/key-statistics/', 'https://finance.yahoo.com/quote/BBVA.MC/key-statistics/', 'https://finance.yahoo.com/quote/BKR/key-statistics/', 'https://finance.yahoo.com/quote/APLE/key-statistics/', 'https://finance.yahoo.com/quote/MTCH/key-statistics/', 'https://finance.yahoo.com/quote/JWN/key-statistics/', 'https://finance.yahoo.com/quote/URBN/key-statistics/', 'https://finance.yahoo.com/quote/ARCC/key-statist

In [11]:
#float(statslst[37])
sirdf = pd.DataFrame(sir_data)

max_transaction = top3_instrans_alltickers[["symbol","transaction_value"]].groupby("symbol").max().reset_index()

max_transaction.rename(columns={'symbol': 'Ticker'}, inplace=True)


In [12]:
#I will get sentiment from top3 rescent articles
# I will avg the sentiment score
# I will obtain label off of sentiment score

In [13]:
combinedf = sirdf.merge(right = max_transaction, how = "inner", on ="Ticker")


def rank_trans(x):
   
    if x <200000:
        return "small transaction"
    if x>=200000 and x<1000000:
        return "medium transaction"
    if x>=1000000:
        return "large transaction"

combinedf["transaction_value"] = combinedf["transaction_value"].apply(rank_trans)
tckrlst = combinedf["Ticker"].tolist()
combinedf


,Ticker,Short_Interest_Rate,transaction_value
0,SLM,6.11,medium transaction
1,AEO,3.45,large transaction
2,MGM,4.13,large transaction
3,CVS,2.21,large transaction
4,FOXA,8.97,large transaction
5,ACGL,4.28,large transaction
6,CVE.TO,2.17,large transaction
7,SYF,4.99,large transaction
8,DAL,4.04,large transaction
9,CHX,6.12,medium transaction


In [14]:

def scrape_articles(top3articles,tic):
 sentiment_scores =[]
 mean_sent_scr = None
 for i in top3articles:
     i["ticker_sentiment"]
     a = i["ticker_sentiment"]
     for j in a:
         if j["ticker"] == tic and float(j["relevance_score"]) > 0.5:
            sentiment_scores.append(float(j["ticker_sentiment_score"]))
            mean_sent_scr = sum(sentiment_scores)/len(sentiment_scores)
 if mean_sent_scr is not None:
  if mean_sent_scr <= -0.35:
     return "Bearish"
  if mean_sent_scr > -0.35 and mean_sent_scr <= -0.15:
     return "Somewhat-Bearish"
  if mean_sent_scr > -0.15 and mean_sent_scr < 0.15:
     return "Neutral"
  if mean_sent_scr >= -0.15 and mean_sent_scr < 0.35:
     return "Somewhat-Bullish"
  if mean_sent_scr >=0.35:
     return "Bullish"
 else:
    return "low relevance score"


In [15]:
print(two_months)

20240805T1113


In [3]:
path_to_api = '/Users/lukasamare/data'
filename = 'alphavantagekey.txt'
api_key_path = os.path.join(path_to_api,filename)
with open(api_key_path, 'r') as file:
 alphavantagekey = file.read().strip()
alphavantagekey

'IMYD2DG739BJVB81'

In [16]:
#alphaquerey = "https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=AAPL&time_from="+two_months+"&sort=LATEST&limit=2&apikey=SAIGXA9UVDHR0EAQ"
# r  = requests.get(alphaquerey)
# data = r.json()
#top3articles  = data["feed"][:3]
ticker_sentiment =[]

for tic in tckrlst:
    if "." not in tic:
     print(tic)
     query = "https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers="+tic+"&time_from="+two_months+"&sort=LATEST&apikey="+alphavantagekey
     r  = requests.get(query)
     data = r.json()
     top3articles  = data["feed"][:3]
     sentiment = {"Ticker":tic}
     sentiment['sentiment'] =  scrape_articles(top3articles,tic)
     ticker_sentiment.append(sentiment)
     time.sleep(45)  # Wait 12 seconds before the next request
    else:
     print("out of market"+tic)
     sentiment = {"Ticker":tic}
     sentiment['sentiment'] =  "out of market"
     ticker_sentiment.append(sentiment)

masterdf =pd.DataFrame(ticker_sentiment)
data

    



SLM
AEO
MGM
CVS
FOXA
ACGL
out of marketCVE.TO
SYF
DAL
CHX
out of marketELD.TO
out of marketBBVA.MC
BKR
APLE
MTCH
JWN
URBN
ARCC
FNF


{'items': '8',
 'sentiment_score_definition': 'x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish',
 'relevance_score_definition': '0 < x <= 1, with a higher score indicating higher relevance.',
 'feed': [{'title': '5 Mid-Cap Value Mutual Funds for Long-Term Gains',
   'url': 'https://www.zacks.com/stock/news/2346096/5-mid-cap-value-mutual-funds-for-long-term-gains',
   'time_published': '20241004T130800',
   'authors': ['Zacks Equity Research'],
   'summary': 'Invest in mid-cap value mutual funds like TRMCX, VASVX, MVCKX, FVLKX and TRVPX for long-term gains.',
   'banner_image': 'https://staticx-tuner.zacks.com/images/articles/main/6d/19169.jpg',
   'source': 'Zacks Commentary',
   'category_within_source': 'n/a',
   'source_domain': 'www.zacks.com',
   'topics': [{'topic': 'Life Sciences', 'relevance_score': '0.166667'},
    {'topic': 'Energy & Transportation', 'relevance_score': '0.166667'},
  

In [17]:
#masterdf

# tic_prac = tvh_ticker_lst[10:]
# print(data)
# print(tvh_ticker_lst[0])
# print(two_months)
# for i in tic_prac:
#  query = "https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=BBVA.MC&time_from=20240802T2228&sort=LATEST&limit=2&apikey=3MRHQSIWXVKRB0H1" 
#  r  = requests.get(query)
#  data = r.json()
#  data["feed"][:3]
print(data)
print(query)
tckrlst = combinedf["Ticker"].tolist()

print(tckrlst)
# combinedf
# top3_instrans_alltickers.set_index("symbol")

{'items': '8', 'sentiment_score_definition': 'x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish', 'relevance_score_definition': '0 < x <= 1, with a higher score indicating higher relevance.', 'feed': [{'title': '5 Mid-Cap Value Mutual Funds for Long-Term Gains', 'url': 'https://www.zacks.com/stock/news/2346096/5-mid-cap-value-mutual-funds-for-long-term-gains', 'time_published': '20241004T130800', 'authors': ['Zacks Equity Research'], 'summary': 'Invest in mid-cap value mutual funds like TRMCX, VASVX, MVCKX, FVLKX and TRVPX for long-term gains.', 'banner_image': 'https://staticx-tuner.zacks.com/images/articles/main/6d/19169.jpg', 'source': 'Zacks Commentary', 'category_within_source': 'n/a', 'source_domain': 'www.zacks.com', 'topics': [{'topic': 'Life Sciences', 'relevance_score': '0.166667'}, {'topic': 'Energy & Transportation', 'relevance_score': '0.166667'}, {'topic': 'Technology', 'relevance_s

In [18]:
print("before"+ masterdf)
masterdf = combinedf.merge(right = masterdf, on="Ticker")
masterdf

           Ticker                  sentiment
0       beforeSLM  beforelow relevance score
1       beforeAEO              beforeNeutral
2       beforeMGM  beforelow relevance score
3       beforeCVS  beforelow relevance score
4      beforeFOXA  beforelow relevance score
5      beforeACGL              beforeNeutral
6    beforeCVE.TO        beforeout of market
7       beforeSYF  beforelow relevance score
8       beforeDAL  beforelow relevance score
9       beforeCHX              beforeBullish
10   beforeELD.TO        beforeout of market
11  beforeBBVA.MC        beforeout of market
12      beforeBKR              beforeBullish
13     beforeAPLE  beforelow relevance score
14     beforeMTCH  beforelow relevance score
15      beforeJWN  beforelow relevance score
16     beforeURBN     beforeSomewhat-Bullish
17     beforeARCC     beforeSomewhat-Bullish
18      beforeFNF  beforelow relevance score


,Ticker,Short_Interest_Rate,transaction_value,sentiment
0,SLM,6.11,medium transaction,low relevance score
1,AEO,3.45,large transaction,Neutral
2,MGM,4.13,large transaction,low relevance score
3,CVS,2.21,large transaction,low relevance score
4,FOXA,8.97,large transaction,low relevance score
5,ACGL,4.28,large transaction,Neutral
6,CVE.TO,2.17,large transaction,out of market
7,SYF,4.99,large transaction,low relevance score
8,DAL,4.04,large transaction,low relevance score
9,CHX,6.12,medium transaction,Bullish


In [19]:
filtered_master = masterdf[((masterdf["transaction_value"]=="medium transaction")|(masterdf["transaction_value"]=="large transaction"))&(masterdf["Short_Interest_Rate"]>5)&((masterdf["sentiment"]=="Somewhat-Bullish")|(masterdf["sentiment"]=="Bullish"))]
filtered_master

,Ticker,Short_Interest_Rate,transaction_value,sentiment
9,CHX,6.12,medium transaction,Bullish
17,ARCC,5.73,medium transaction,Somewhat-Bullish


In [20]:
#practice on 1 ticker
#BKR
# import requests
# from datetime import datetime, timedelta
# ticker_sentiment =[]
# def scrape_articles(top3articles):
#  sentiment_scores =[]
#  mean_sent_scr = None
#  for i in top3articles:
#      i["ticker_sentiment"]
#      a = i["ticker_sentiment"]
#      for j in a:
#          if j["ticker"] == ticker and float(j["relevance_score"]) > 0.5:
#             sentiment_scores.append(float(j["ticker_sentiment_score"]))
#             mean_sent_scr = sum(sentiment_scores)/len(sentiment_scores)
#  if mean_sent_scr is not None:
#   if mean_sent_scr <= -0.35:
#      return "Bearish"
#   if mean_sent_scr > -0.35 and mean_sent_scr <= -0.15:
#      return "Somewhat-Bearish"
#   if mean_sent_scr > -0.15 and mean_sent_scr < 0.15:
#      return "Neutral"
#   if mean_sent_scr >= -0.15 and mean_sent_scr < 0.35:
#      return "Somewhat-Bullish"
#   if mean_sent_scr >=0.35:
#      return "Bullish"
#  else:
#     return "low relevance score"

# today = datetime.today()

# two_months_ago = today-timedelta(days=60)

# two_months = str(two_months_ago).replace("-","").replace(" ","T")[:14].replace(":","")

# for ticker in ["BBVA.MC"]:
#     query = "https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers="+ticker+"&time_from="+two_months+"&sort=LATEST&limit=2&apikey=MIQ8OGJYT5MV8L92"
#     r  = requests.get(query)
#     data = r.json()
#     top3articles  = data["feed"][:3]
#     sentiment = {"Ticker":ticker}
#     sentiment['sentiment'] =  scrape_articles(top3articles)
#     ticker_sentiment.append(sentiment)


In [21]:
# print(ticker_sentiment)
# masterdf =pd.DataFrame(ticker_sentiment)
# masterdf = combinedf.merge(right = masterdf, on="Ticker")
# # masterdf

# print(data)
# print(tvh_ticker_lst)
# len(ticker_lst)
# query = "https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=FOXA&sort=LATEST&apikey=3MRHQSIWXVKRB0H1"
# r  = requests.get(query)
# data = r.json()
# top3articles  = data["feed"][:3]


In [22]:
# #BKR
# import requests
# from datetime import datetime, timedelta
# lst = ["BKR"]
# len(lst)
# today = datetime.today()

# two_months_ago = today-timedelta(days=60)

# two_months = str(two_months_ago).replace("-","").replace(" ","T")[:14].replace(":","")
# query = "https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=BKR&time_from="+two_months+"&sort=LATEST&limit=2&apikey=SAIGXA9UVDHR0EAQ"
# r  = requests.get(query)
# data = r.json()
# top3articles  = data["feed"][:3]
# top3articles
data["feed"]

[{'title': '5 Mid-Cap Value Mutual Funds for Long-Term Gains',
  'url': 'https://www.zacks.com/stock/news/2346096/5-mid-cap-value-mutual-funds-for-long-term-gains',
  'time_published': '20241004T130800',
  'authors': ['Zacks Equity Research'],
  'summary': 'Invest in mid-cap value mutual funds like TRMCX, VASVX, MVCKX, FVLKX and TRVPX for long-term gains.',
  'banner_image': 'https://staticx-tuner.zacks.com/images/articles/main/6d/19169.jpg',
  'source': 'Zacks Commentary',
  'category_within_source': 'n/a',
  'source_domain': 'www.zacks.com',
  'topics': [{'topic': 'Life Sciences', 'relevance_score': '0.166667'},
   {'topic': 'Energy & Transportation', 'relevance_score': '0.166667'},
   {'topic': 'Technology', 'relevance_score': '0.166667'},
   {'topic': 'Finance', 'relevance_score': '0.166667'},
   {'topic': 'Economy - Monetary', 'relevance_score': '0.576289'},
   {'topic': 'Financial Markets', 'relevance_score': '0.723405'},
   {'topic': 'Manufacturing', 'relevance_score': '0.166667

In [25]:
# def scrape_articles(top3articles):
#  sentiment_scores =[]
#  mean_sent_scr = None
#  for i in top3articles:
#      i["ticker_sentiment"]
#      a = i["ticker_sentiment"]
#      for j in a:
#          if j["ticker"] == ticker and float(j["relevance_score"]) > 0.5:
#             sentiment_scores.append(float(j["ticker_sentiment_score"]))
#             mean_sent_scr = sum(sentiment_scores)/len(sentiment_scores)
#  if mean_sent_scr is not None:
#   if mean_sent_scr <= -0.35:
#      return "Bearish"
#   if mean_sent_scr > -0.35 and mean_sent_scr <= -0.15:
#      return "Somewhat-Bearish"
#   if mean_sent_scr > -0.15 and mean_sent_scr < 0.15:
#      return "Neutral"
#   if mean_sent_scr >= -0.15 and mean_sent_scr < 0.35:
#      return "Somewhat-Bullish"
#   if mean_sent_scr >=0.35:
#      return "Bullish"
#  else:
#     return "low relevance score"
# print(scrape_articles(top3articles))
combinedf

,Ticker,Short_Interest_Rate,transaction_value
0,SLM,6.11,medium transaction
1,AEO,3.45,large transaction
2,MGM,4.13,large transaction
3,CVS,2.21,large transaction
4,FOXA,8.97,large transaction
5,ACGL,4.28,large transaction
6,CVE.TO,2.17,large transaction
7,SYF,4.99,large transaction
8,DAL,4.04,large transaction
9,CHX,6.12,medium transaction


In [24]:
filtered_master.to_csv("stocks_meet_criteria.csv",index=False)
combinedf.to_csv("all_stocks.csv",index=False)